# Circuit optimization of QFT3

This notebook shows an example execution of the simplification algorithms detailed in my bachelor thesis applied to a Quantum Fourier Transform on 3 qubits.

First, we have to import the modules we will need.
The algorithm will be performed by `pyzx`, and we will use `qiskit` to visualize the quantum circuits.

In [1]:
import sys
import math
import pyzx as zx
from qiskit import QuantumCircuit

Following is QASM code describing QFT3.
Printed afterwards is this code as a diagrammatic quantum circuit.

Note that QASM 2.0 does not support controlled phase gates, therefore those are replaced by universal gate constructions as in Example 2.4.2.8.

In [2]:
qasm = """OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
h q[0];
rz(pi/4) q[1];
CX q[1], q[0];
rz(-pi/4) q[0];
CX q[1], q[0];
rz(pi/4) q[0];
rz(pi/8) q[2];
CX q[2], q[0];
rz(-pi/8) q[0];
CX q[2], q[0];
rz(pi/8) q[0];
h q[1];
rz(pi/4) q[2];
CX q[2], q[1];
rz(-pi/4) q[1];
CX q[2], q[1];
rz(pi/4) q[1];
h q[2];
CX q[0], q[2];
CX q[2], q[0];
CX q[0], q[2];"""

circuit = zx.Circuit.from_qasm(qasm)
qc = QuantumCircuit.from_qasm_str(qasm)
qc.draw()

┌───┐   ┌───┐┌──────────┐┌───┐┌─────────┐┌───┐┌──────────┐┌───┐»
q_0: ───┤ H ├───┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ X ├┤ Rz(-π/8) ├┤ X ├»
     ┌──┴───┴──┐└─┬─┘└──────────┘└─┬─┘└──┬───┬──┘└─┬─┘└──────────┘└─┬─┘»
q_1: ┤ Rz(π/4) ├──■────────────────■─────┤ H ├─────┼────────────────┼──»
     ├─────────┤                         └───┘     │                │  »
q_2: ┤ Rz(π/8) ├───────────────────────────────────■────────────────■──»
     └─────────┘                                                       »
«     ┌─────────┐                                      ┌───┐     
«q_0: ┤ Rz(π/8) ├───────────────────────────────────■──┤ X ├──■──
«     └─────────┘┌───┐┌──────────┐┌───┐┌─────────┐  │  └─┬─┘  │  
«q_1: ───────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├──┼────┼────┼──
«     ┌─────────┐└─┬─┘└──────────┘└─┬─┘└──┬───┬──┘┌─┴─┐  │  ┌─┴─┐
«q_2: ┤ Rz(π/4) ├──■────────────────■─────┤ H ├───┤ X ├──■──┤ X ├
«     └─────────┘                         └───┘   └───┘     └───┘

Let's look at some stats about this circuit:

In [3]:
print("T-count: ", circuit.tcount())
print("2-qubit-count: ", circuit.twoqubitcount())
print("Gate count: ", len(circuit.gates))

T-count:  9
2-qubit-count:  9
Gate count:  21


Our goal will be to optimize this circuit, i.e. bringing those numbers down as much as possible, especially gate count (less gates = less effort to implement) and *T*-count (those are specifically expensive gates to implement, see Definition 2.4.6).

For this purpose, we will translate this circuit into a ZX diagram as in § 4.4.

In [4]:
g = circuit.copy().to_basic_gates().to_graph();
zx.draw(g)

Now, we can start the optimization algorithm from Theorem 6.0.1.

The following algorithm is an interactive version taken from [demos/gettingstarted.ipynb in the `pyzx` GitHub repo](https://github.com/Quantomatic/pyzx/blob/9722f783e7e2aab025220d537bfd75454973da71/demos/gettingstarted.ipynb) (Apache License 2.0).

You can cleary follow the steps taken in the algorithm by moving the slider, first observing the transformation into a graph-like ZX diagram as in Lemma 4.5.2, then the simplification, more specifically in this case the pivots from Lemma 5.1.5 and Remark 5.1.6.

In [5]:
from ipywidgets import widgets
from IPython.display import display, Markdown

from pyzx.simplify import clifford_iter

graph = g.copy()
graphs = [zx.draw_matplotlib(graph)]
glist = []
names = ["start"]
for gs, n in clifford_iter(graph):
    glist.append(gs)
    graphs.append(zx.draw_matplotlib(gs))
    names.append(n)
zx.drawing.pack_circuit_nf(gs,'grg')
graphs.append(zx.draw_matplotlib(gs))
names.append("reposition")
g1 = gs.copy()

def plotter(rewrite):
    display(Markdown("Rewrite step: " + names[rewrite]))
    display(graphs[rewrite])

w = widgets.interactive(plotter, rewrite=(0,len(graphs)-1))
slider = w.children[0]
slider.layout.width = "{!s}px".format(min(800,50*len(graphs)))
output = w.children[-1]
output.layout.height = "{!s}px".format(200+3*20)
slider.value = 0
w

interactive(children=(IntSlider(value=0, description='rewrite', layout=Layout(width='500px'), max=9), Output(l…

The end result of this algorithm is this ZX diagram:

In [6]:
g1.normalize()
zx.draw(g1)

Now that we have a reduced ZX diagram, we need to get it back into a form that we can read as a quantum circuit.
Therefore, we initiate the extraction procedure from Theorem 6.2.4:

In [7]:
c = zx.extract_circuit(g1.copy())
zx.draw(c)

And finally translate those gates back into quantum circuit notation.

In [8]:
qc = QuantumCircuit.from_qasm_str(c.to_qasm())
qc.draw()

┌───┐     ┌───┐┌───┐                                    »
q_0: ───────■───────┤ X ├──■──┤ H ├┤ H ├────────────────────────────────────»
     ┌───┐  │  ┌───┐└─┬─┘  │  └───┘└───┘                                    »
q_1: ┤ H ├──┼──┤ H ├──┼────┼────■─────────────────────────■─────────────────»
     └───┘┌─┴─┐└───┘  │  ┌─┴─┐  │  ┌───┐┌──────────┐┌───┐ │ ┌───┐┌─────────┐»
q_2: ─────┤ X ├───────■──┤ X ├──■──┤ H ├┤ Rz(7π/4) ├┤ H ├─■─┤ H ├┤ Rz(π/4) ├»
          └───┘          └───┘     └───┘└──────────┘└───┘   └───┘└─────────┘»
«                                                                   »
«q_0: ──────■───■─────────────────────────────────■───────────■─────»
«           │   │   ┌─────────┐ ┌───┐┌──────────┐ │ ┌───┐     │     »
«q_1: ──────┼───■───┤ Rz(π/4) ├─┤ H ├┤ Rz(7π/4) ├─┼─┤ H ├─────■─────»
«     ┌───┐ │ ┌───┐┌┴─────────┴┐├───┤└──────────┘ │ ├───┤┌─────────┐»
«q_2: ┤ H ├─■─┤ H ├┤ Rz(15π/8) ├┤ H ├─────────────■─┤ H ├┤ Rz(π/8) ├»
«     └───┘   └───┘└───────────┘└───┘               └───┘└─────────┘»
«     ┌──────────┐   ┌───┐   
«q_0: ┤ Rz(3π/8) ├───┤ H ├───
«     └──┬───┬───┘┌──┴───┴──┐
«q_1: ───┤ H ├────┤ Rz(π/4) ├
«        └───┘    └─────────┘
«q_2: ───────────────────────
«

Now it is time to check what we have gained by applying this procedure.

In [9]:
print("Are tensors equal? ", zx.compare_tensors(c,g,preserve_scalar=False))
print("T-count: ", c.tcount())
print("2-qubit-count: ", c.twoqubitcount())
print("Gate count: ", len(c.gates))

Are tensors equal?  True
T-count:  8
2-qubit-count:  9
Gate count:  30


While we did in fact reduce the *T*-count by one, the total gate count exploded to 30.

Careful observers will notice that the visualized circuit above is obviously not optimal:
There are, among other things, double Hadamard gates that could be canceled with each other.

Therefore, we need to do an additional pass of "classic" optimization techniques after the ZX optimization procedure.

In [10]:
c_o = c.copy()
c_o = zx.optimize.basic_optimization(c_o.to_basic_gates())

In [11]:
qc_o = QuantumCircuit.from_qasm_str(c_o.to_qasm())
qc_o.draw()

┌───┐   ┌──────────┐   ┌───┐    ┌─────────┐                 »
q_0: ─■────┤ H ├───┤ Rz(7π/4) ├───┤ X ├────┤ Rz(π/4) ├─────────────────»
      │ ┌──┴───┴──┐└──────────┘   └─┬─┘    └──┬───┬──┘┌──────────┐┌───┐»
q_1: ─■─┤ Rz(π/4) ├─────■───────────■─────────┤ H ├───┤ Rz(7π/4) ├┤ X ├»
        └─────────┘     │      ┌──────────┐   └───┘   └──────────┘└─┬─┘»
q_2: ───────────────────■──────┤ Rz(3π/8) ├─────────────────────────■──»
                               └──────────┘                            »
«                ┌───┐┌───────────┐┌───┐┌─────────┐     ┌───┐     
«q_0: ───────────┤ X ├┤ Rz(15π/8) ├┤ X ├┤ Rz(π/8) ├──■──┤ X ├──■──
«     ┌─────────┐└─┬─┘└───────────┘└─┬─┘└─────────┘  │  └─┬─┘  │  
«q_1: ┤ Rz(π/4) ├──┼─────────────────┼───────────────┼────┼────┼──
«     └─────────┘  │                 │     ┌───┐   ┌─┴─┐  │  ┌─┴─┐
«q_2: ─────────────■─────────────────■─────┤ H ├───┤ X ├──■──┤ X ├
«                                          └───┘   └───┘     └───┘

In [12]:
print("Are tensors equal? ", zx.compare_tensors(c_o,g,preserve_scalar=False))
print("T-count: ", c_o.tcount())
print("2-qubit-count: ", c_o.twoqubitcount())
print("Gate count: ", len(c_o.gates))

Are tensors equal?  True
T-count:  8
2-qubit-count:  9
Gate count:  20


Now we can see that we reduced both the *T*-count and the total gate count by one.

Let's compare the result with just taking the circuit and applying the basic optimization from the last step, without the simplification of Theorem 6.0.1.

In [13]:
c_optimized = zx.optimize.basic_optimization(circuit.copy().to_basic_gates())

print("Are tensors equal? ", zx.compare_tensors(c_optimized,g,preserve_scalar=False))
print("T-count: ", c_optimized.tcount())
print("2-qubit-count: ", c_optimized.twoqubitcount())
print("Gate count: ", len(c_optimized.gates))

Are tensors equal?  True
T-count:  8
2-qubit-count:  9
Gate count:  20


Interestingly, this is the exact same result - _T_-count and total gate count was reduced by one.
This means that QFT3 is not affected by the simplification from Theorem 6.0.1 (though, arguably, it also doesn't make it worse other than wasting computational resources).

But the next question is: Does this mean QFT3 is already optimal, or can it be improved further?
To answer this, we will now apply the full algorithm from Theorem 6.0.7, and apply it to the QFT3 circuit.

In [14]:
g_f = circuit.copy().to_basic_gates().to_graph()
zx.full_reduce(g_f)
g_f.normalize()

This yields the following ZX diagram.
The phase gadgets that were introduced by the algorithm can clearly be seen, making this result very different from the result from Theorem 6.0.1.

In [15]:
zx.draw(g_f)

But is it *better* than the previous result in terms of gate counts?
Let's extract it into a circuit, run our basic optimization, and see.

In [16]:
c_f = zx.extract_circuit(g_f.copy())
c_fo = zx.optimize.basic_optimization(c_f.to_basic_gates())

In [17]:
qc_f = QuantumCircuit.from_qasm_str(c_fo.to_qasm())
qc_f.draw()

┌───┐   ┌─────────┐                                     ┌───┐    »
q_0: ─■────┤ H ├───┤ Rz(π/4) ├──■────────────────────────────■─────┤ X ├────»
      │ ┌──┴───┴──┐└─────────┘  │                            │     └─┬─┘    »
q_1: ─■─┤ Rz(π/4) ├─────■───────┼────────────────■───────────┼───────■──────»
        └─────────┘   ┌─┴─┐   ┌─┴─┐┌──────────┐┌─┴─┐┌─────┐┌─┴─┐┌──────────┐»
q_2: ─────────────────┤ X ├───┤ X ├┤ Rz(3π/8) ├┤ X ├┤ Sdg ├┤ X ├┤ Rz(3π/8) ├»
                      └───┘   └───┘└──────────┘└───┘└─────┘└───┘└──────────┘»
«     ┌───────────┐                                 ┌───┐     
«q_0: ┤ Rz(15π/8) ├──────────────────────────────■──┤ X ├──■──
«     └───┬───┬───┘┌──────────┐┌───┐┌─────────┐  │  └─┬─┘  │  
«q_1: ────┤ H ├────┤ Rz(7π/4) ├┤ X ├┤ Rz(π/4) ├──┼────┼────┼──
«         └───┘    └──────────┘└─┬─┘└──┬───┬──┘┌─┴─┐  │  ┌─┴─┐
«q_2: ───────────────────────────■─────┤ H ├───┤ X ├──■──┤ X ├
«                                      └───┘   └───┘     └───┘

In [18]:
print("Are tensors equal? ", zx.compare_tensors(c_fo,g,preserve_scalar=False))
print("T-count: ", c_fo.tcount())
print("2-qubit-count: ", c_fo.twoqubitcount())
print("Gate count: ", len(c_fo.gates))

Are tensors equal?  True
T-count:  7
2-qubit-count:  10
Gate count:  21


And indeed, we now have improved _T_-count by one more gate! At the cost of introducing one additional 2-qubit gate. As our goal is mainly to reduce the usually expensive _T_-count, this is acceptable.